#### Decision tree for allocating oil type to barge traffic
see google drawing (Barge_Oil_Attribution)[https://docs.google.com/drawings/d/10PM53-UnnILYCAPKU9MxiR-Y4OW0tIMhVzSjaHr-iSc/edit ] for a visual representation

In [1]:
# These will become function inputs
destination = 'US'
origin = 'Pacific'
ship_type = 'barge'
random_seed=None

# This will become an output that flags whether tug 
# is deemed non-oil cargo and, hence, fuel-spill risk
fuel_flag = 0

In [2]:
import numpy 
import yaml
import pathlib

In [3]:
# load the list of CAD, US and generic origins and destinations
master_dir  = '/Users/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/monte_carlo/'
master_file = 'master.yaml'

with open(f'{master_dir}{master_file}') as file:
    master = yaml.safe_load(file)

# Assign US and CAD origin/destinations from master file
CAD_origin_destination = master['categories']['CAD_origin_destination']
US_origin_destination = master['categories']['US_origin_destination']

# Get file paths to fuel-type yaml files
home = pathlib.Path(master['directories'])
CAD_yaml     = home/master['files']['CAD_origin']
WA_in_yaml   = home/master['files']['WA_destination']
WA_out_yaml  = home/master['files']['WA_origin']
US_yaml      = home/master['files']['US_origin']
USall_yaml   = home/master['files']['US_combined']
Pacific_yaml = home/master['files']['Pacific_origin']
fuel_yaml    = home/master['files']['fuel']

In [4]:
with fuel_yaml.open("rt") as file:
     cargo = yaml.safe_load(file)

In [5]:
cargo['barge']

{'bunker': 0.38, 'diesel': 0.62}

#### Create decision-making tree for barge traffic

In [6]:
def get_oil_type_cargo(yaml_file, facility):
     with yaml_file.open("rt") as file:
          ships = yaml.safe_load(file)
          ship = ships[facility][ship_type]
          probability = [ship[fuel]['fraction_of_total'] for fuel in ship]    
          fuel_type = random_generator.choice(list(ship.keys()), p = probability)
          return fuel_type
        
def get_oil_type_fuel(yaml_file, ship_type):
     with yaml_file.open("rt") as file:
         fuel  = yaml.safe_load(file)
         barge = fuel['barge']
         probability = [barge[fuel] for fuel in barge]    
         fuel_type = random_generator.choice(list(barge.keys()), p = probability)
         return fuel_type         

In [7]:
# these pairs need to be used together for "get_oil_type_cargo" (but don't yet have error-checks in place):
# - "WA_in_yaml" and "destination"
# - "WA_out_yaml" and "origin"

# Need to add a catch for erroneous cases where origin-destination in AIS analysis
# pairs vessel-type and facility to null values in the DOE transfer data. 
# For these cases (which shouldn't happen but might), we will use the generic US fuel allocations

# Initialize PCG-64 random number generator
random_generator = numpy.random.default_rng(random_seed)

if origin in CAD_origin_destination:
    if origin == 'Westridge Marine Terminal':
        if destination in CAD_origin_destination:
            fuel_type = 'jet'
        else:
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
    else:
        if destination in US_origin_destination:
            # we have better information on WA fuel transfers, so I'm prioritizing this information source
            fuel_type = get_oil_type_cargo(WA_in_yaml, destination)
        elif destination == 'ESSO Nanaimo Departure Bay':
            fuel_type = get_oil_type_cargo(CAD_yaml, destination)
        elif destination == 'Suncor Nanaimo':
            fuel_type = get_oil_type_cargo(CAD_yaml, destination)
        else: 
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
elif origin in US_origin_destination:
    fuel_type = get_oil_type_cargo(WA_out_yaml, origin)
elif destination in US_origin_destination:
    fuel_type = get_oil_type_cargo(WA_in_yaml, destination)
elif destination in CAD_origin_destination:
    if destination == 'Westridge Marine Terminal':
        # Westridge doesn't receive crude for storage 
        fuel_type = 'jet'
    else:
        fuel_type = get_oil_type_cargo(CAD_yaml, destination)
# All remaining cases are deemed non-tank traffic
elif origin == 'Pacific':
    print(origin)
    fuel_flag = 1
    fuel_type = get_oil_type_fuel(fuel_yaml, ship_type)
elif origin == 'US':
    fuel_flag = 1
    fuel_type = get_oil_type_fuel(fuel_yaml, ship_type)
elif origin == 'Canada':
    fuel_flag = 1
    fuel_type = get_oil_type_fuel(fuel_yaml, ship_type)
else:
    fuel_flag = 1
    fuel_type = get_oil_type_fuel(fuel_yaml, ship_type)

Pacific


In [8]:
fuel_flag

1

In [9]:
fuel_type

'bunker'

In [10]:
origin

'Pacific'